In [1]:
# Ensure most recent source code is used (and not cached)
%load_ext autoreload
%autoreload 2

In [3]:
# Import other libraries
import pickle

In [5]:
# Import functions from other python files in the project
from pdga_score_scraper import *

2025-11-03 13:57:28.778 | INFO     | pdga_score_scraper.config:<module>:11 - PROJ_ROOT path is: /home/jon/repos/pdga-score-scraper
